# DAG Sample

This pipeline implements a [directed acyclic graph (DAG)](https://en.wikipedia.org/wiki/Directed_acyclic_graph) of stage functions.
The pipeline has six stages, `A`, `B`, `C`, `D`, `E`, and `F`.
For the purposes of the demo, each stage sleeps for tens of milliseonds and then completes.
Here is the structure of the DAG:

![](../../documentation/dag.png)

This graph represents the following stage function dependencies:
* `B` and `C` can't run until `A` completes.
* `D` depends on `B` and `C`
* `F` depends on `D` and `E`

We'll start by importing the pipeline:

In [1]:
from dag import dag_pipeline_spec

The DAG pipeline doesn't actually read its test cases, but we need to define one empty case so that Gotaglio's `run()` method will invoke the pipeline.

In [ ]:
cases = [
  {
    "uuid": "6919a246-945f-4d93-b39d-d798244dd08e",
  },
]

Instantiate the Gotaglio object that runs the pipeline.

In [3]:
from gotaglio.gotag import Gotaglio

gt = Gotaglio([dag_pipeline_spec])

Run the pipeline. The summarize() method generates a table that shows the execution of the stages over time.
Each row corresponds to a point in time where a stage function started or stopped. The green highlight
indicates when stages were running.

In [5]:
result = gt.run(
  "dag",
  cases,
)

             Timeline for case              
    6f31c4df-bba6-42cd-a3ae-24f31d5503fa    
┏━━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃ step ┃  A  ┃  B  ┃  C  ┃  D  ┃  E  ┃  F  ┃
┡━━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│    1 │  x  │     │     │     │     │     │
│    2 │  x  │     │     │     │  x  │     │
│    3 │  x  │     │     │     │  x  │     │
│    4 │     │     │     │     │  x  │     │
│    5 │     │  x  │     │     │     │     │
│    6 │     │  x  │  x  │     │     │     │
│    7 │     │  x  │  x  │     │     │     │
│    8 │     │     │  x  │     │     │     │
│    9 │     │     │     │  x  │     │     │
│   10 │     │     │     │  x  │     │     │
│   11 │     │     │     │     │     │  x  │
│   12 │     │     │     │     │     │  x  │
└──────┴─────┴─────┴─────┴─────┴─────┴─────┘

We can rerun this example with a different model, this time the `flakey` mock, which alternately returns the correct answer, returns "hello world", and raises an exception. The mock models are good for debugging new pipelines locally, before connecting to a real model.

In [5]:
result2 = gt.rerun(
  result,
  {
    "infer.model.name": "flakey"
  },
  save=True
)
gt.format(result2)

             Summary for 8236c48e-ce5a-4f9d-9310-a09a9d957570              
┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃     id ┃ status   ┃ cost ┃ keywords    ┃ user                           ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 6f3.00 │ COMPLETE │ 0.00 │ numbers     │ 1+1                            │
│ ceb.00 │ ERROR    │      │ text        │ one hundred two divided by two │
│ 178.00 │ ERROR    │      │ hexidecimal │ ff + a                         │
└────────┴──────────┴──────┴─────────────┴────────────────────────────────┘

Total: 3
Complete: 1/3 (33.33%)
Error: 2/3 (66.67%)
Passed: 1/3 (33.33%)
Failed: 0/3 (0.00%)


Results written to logs\8236c48e-ce5a-4f9d-9310-a09a9d957570.json


## Run: 8236c48e-ce5a-4f9d-9310-a09a9d957570
## Case: 6f3 - PASSED
**Keywords:** numbers  


**system:**
You are a desktop calculator that computes the value of mathematical expressions.
The input is base 10.
Your output should be just a base 10 numerical result.

**user:** _1+1_

**assistant:**
2.0


## Case: ceb - FAILED
**Keywords:** text  


### Turn 1: **ERROR**  
Error: Context: Extracting numerical answer from LLM response.
Error: could not convert string to float: 'hello world'

~~~
Traceback: Traceback (most recent call last):
  File "C:\git\llm-tools\gotaglio\gotaglio\director2.py", line 145, in process_one_case
    await run_dag(dag, result)
  File "C:\git\llm-tools\gotaglio\gotaglio\dag.py", line 129, in run_dag
    (name, result) = task.result()
                     ^^^^^^^^^^^^^
  File "C:\Users\mhop\AppData\Local\Programs\Python\Python312\Lib\asyncio\futures.py", line 202, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "C:\Users\mhop\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in 
__step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\git\llm-tools\gotaglio\gotaglio\dag.py", line 95, in run_task
    raise e
  File "C:\git\llm-tools\gotaglio\gotaglio\dag.py", line 87, in run_task
    result = await dag["function"](context)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\git\llm-tools\gotaglio\samples2\calc\calc.py", line 100, in extract
    return float(context["stages"]["infer"])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: 'hello world'

Time: 2025-08-13 21:56:22.801577+00:00
~~~
## Case: 178 - FAILED
**Keywords:** hexidecimal  


### Turn 1: **ERROR**  
Error: Error: Flakey model failed

~~~
Traceback: Traceback (most recent call last):
  File "C:\git\llm-tools\gotaglio\gotaglio\director2.py", line 145, in process_one_case
    await run_dag(dag, result)
  File "C:\git\llm-tools\gotaglio\gotaglio\dag.py", line 129, in run_dag
    (name, result) = task.result()
                     ^^^^^^^^^^^^^
  File "C:\Users\mhop\AppData\Local\Programs\Python\Python312\Lib\asyncio\futures.py", line 202, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "C:\Users\mhop\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in 
__step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\git\llm-tools\gotaglio\gotaglio\dag.py", line 95, in run_task
    raise e
  File "C:\git\llm-tools\gotaglio\gotaglio\dag.py", line 87, in run_task
    result = await dag["function"](context)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\git\llm-tools\gotaglio\samples2\calc\calc.py", line 93, in infer
    return await model.infer(context["stages"]["prepare"], context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\git\llm-tools\gotaglio\gotaglio\mocks.py", line 27, in infer
    raise Exception("Flakey model failed")
Exception: Flakey model failed

Time: 2025-08-13 21:56:22.804577+00:00
~~~

Note that we can use the term `"latest"` to refer to the most recently generated runlog.
We could also pass the first few characters of a run's UUID or we could pass the result
object (e.g. `result` or `result2`) directly.

In [6]:
gt.summarize("latest")

             Summary for 8236c48e-ce5a-4f9d-9310-a09a9d957570              
┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃     id ┃ status   ┃ cost ┃ keywords    ┃ user                           ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 6f3.00 │ COMPLETE │ 0.00 │ numbers     │ 1+1                            │
│ ceb.00 │ ERROR    │      │ text        │ one hundred two divided by two │
│ 178.00 │ ERROR    │      │ hexidecimal │ ff + a                         │
└────────┴──────────┴──────┴─────────────┴────────────────────────────────┘

Total: 3
Complete: 1/3 (33.33%)
Error: 2/3 (66.67%)
Passed: 1/3 (33.33%)
Failed: 0/3 (0.00%)


